In [59]:
import os
import pandas as pd
import json
import urllib
import numpy as np
from sklearn import model_selection

In [15]:
# loads the list of xenocanto.com records to full_xeno.csv

birds_path = 'https://raw.githubusercontent.com/microfaune/microfaune/master/oiseaux.csv'
birds = pd.read_csv(birds_path, delimiter = ';')
full_xeno = pd.DataFrame()

for i in range(birds.shape[0]):
    bird_query = birds.iloc[i]["Nom latin"].replace(" ", "+").lower()
    bird_name = birds.iloc[i]["Nom latin"].replace(" ", "_").lower()
    
    url = f'https://www.xeno-canto.org/api/2/recordings?query={bird_query}'
    r = urllib.request.urlopen(url)
    
    bird_data = json.loads(r.read().decode(r.info().get_param('charset') or 'utf-8'))
    
    full_xeno = full_xeno.append(pd.json_normalize(bird_data['recordings']))

full_xeno.insert(loc = 1, column = 'species', value = full_xeno['gen'] + ' ' + full_xeno['sp'])
full_xeno.insert(loc = 2, column = 'train/dev_set', value = '')

full_xeno.to_csv(r'/content/drive/My Drive/xenocanto_data/full_xenocanto.csv', index = False)

In [2]:
# loading full_xeno from the already existing full_xeno.csv
full_xeno = pd.read_csv('/content/drive/My Drive/xenocanto_data/full_xenocanto.csv') # le 26/10/2020 23948 enregistrements

In [45]:
full_xeno.head()

,id,species,train/dev_set,gen,sp,ssp,en,rec,cnt,loc,lat,lng,alt,type,url,file,file-name,lic,q,length,time,date,uploaded,also,rmk,bird-seen,playback-used,sono.small,sono.med,sono.large,sono.full
0,597996,Prunella modularis,,Prunella,modularis,NaN,Dunnock,Uku Paal,Estonia,"Tähtvere Parish, Tartu maakond",58.3971,26.5377,30,alarm call,//www.xeno-canto.org/597996,//www.xeno-canto.org/597996/download,XC597996-PRUMOD Ilmatsalu 201026_1925.mp3,//creativecommons.org/licenses/by-nc-sa/4.0/,A,0:31,09:00,2020-10-26,2020-10-26,[''],"HPF, NR",yes,yes,//www.xeno-canto.org/sounds/uploaded/XIBLXHRPJ...,//www.xeno-canto.org/sounds/uploaded/XIBLXHRPJ...,//www.xeno-canto.org/sounds/uploaded/XIBLXHRPJ...,//www.xeno-canto.org/sounds/uploaded/XIBLXHRPJ...
1,597532,Prunella modularis,,Prunella,modularis,NaN,Dunnock,Alexander Henderson,United Kingdom,"Blackdown, Mendip Hills, North Somerset, England",51.3198,-2.7523,190,call,//www.xeno-canto.org/597532,//www.xeno-canto.org/597532/download,XC597532-200620_1000_Blackdown_Dunnock_mp3.mp3,//creativecommons.org/licenses/by-nc-nd/4.0/,A,0:46,08:54,2020-06-15,2020-10-24,"['Turdus merula', 'Troglodytes troglodytes', '...",A Dunnock calling from an exposed Hawthorn tre...,yes,no,//www.xeno-canto.org/sounds/uploaded/HYKVPYDPH...,//www.xeno-canto.org/sounds/uploaded/HYKVPYDPH...,//www.xeno-canto.org/sounds/uploaded/HYKVPYDPH...,//www.xeno-canto.org/sounds/uploaded/HYKVPYDPH...
2,597394,Prunella modularis,,Prunella,modularis,NaN,Dunnock,Alan Dalton,Sweden,"Stockholm, Stockholm Municipality, Stockholms län",59.3158,18.0588,40,"life stage uncertain, nocturnal flight call, s...",//www.xeno-canto.org/597394,//www.xeno-canto.org/597394/download,XC597394-Dunnock 26 oct 0610.mp3,//creativecommons.org/licenses/by-nc-sa/4.0/,A,0:06,20:00,2020-10-23,2020-10-24,[''],NocMig Recording,no,no,//www.xeno-canto.org/sounds/uploaded/BBJSEFYVP...,//www.xeno-canto.org/sounds/uploaded/BBJSEFYVP...,//www.xeno-canto.org/sounds/uploaded/BBJSEFYVP...,//www.xeno-canto.org/sounds/uploaded/BBJSEFYVP...
3,597187,Prunella modularis,,Prunella,modularis,NaN,Dunnock,Simon Elliott,United Kingdom,"Newcastle upon Tyne, Tyne and Wear, England",55.0006,-1.6372,90,"begging call, hatchling or nestling",//www.xeno-canto.org/597187,//www.xeno-canto.org/597187/download,XC597187-Dk-6201D21A.mp3,//creativecommons.org/licenses/by-nc-nd/4.0/,A,1:10,10:00,2012-05-12,2020-10-23,[''],"feeding visits x2, young approx 1w old; very c...",yes,no,//www.xeno-canto.org/sounds/uploaded/FFFADKCCI...,//www.xeno-canto.org/sounds/uploaded/FFFADKCCI...,//www.xeno-canto.org/sounds/uploaded/FFFADKCCI...,//www.xeno-canto.org/sounds/uploaded/FFFADKCCI...
4,597087,Prunella modularis,,Prunella,modularis,NaN,Dunnock,Simon Elliott,United Kingdom,"Newcastle upon Tyne, Tyne and Wear, England",55.0006,-1.6372,90,"begging call, hatchling or nestling",//www.xeno-canto.org/597087,//www.xeno-canto.org/597087/download,XC597087-dunn-D21B.wav,//creativecommons.org/licenses/by-nc-nd/4.0/,A,0:53,10:00,2012-05-12,2020-10-22,['Turdus merula'],"feeding visits x2; very close, mic in nest",yes,no,//www.xeno-canto.org/sounds/uploaded/FFFADKCCI...,//www.xeno-canto.org/sounds/uploaded/FFFADKCCI...,//www.xeno-canto.org/sounds/uploaded/FFFADKCCI...,//www.xeno-canto.org/sounds/uploaded/FFFADKCCI...


In [18]:
# getting species list and records addresses 
species_list = full_xeno.groupby('species', as_index=False).count().sort_values(by  = 'id', ascending = True, ignore_index = True)

In [ ]:
species_list

In [55]:
# pick train and dev examples from full_xeno
for species in species_list['species'] : 
  rand_id = rand_id.append(full_xeno.loc[np.random.choice(full_xeno[full_xeno['species'] == species].index, 138, replace = False)]) # 138 examples (l'espece avec le moins d'examples)

,id,species,train/dev_set,gen,sp,ssp,en,rec,cnt,loc,lat,lng,alt,type,url,file,file-name,lic,q,length,time,date,uploaded,also,rmk,bird-seen,playback-used,sono.small,sono.med,sono.large,sono.full
6444,505641,Turdus viscivorus,,Turdus,viscivorus,NaN,Mistle Thrush,Joost van Bruggen,Netherlands,"Arnhem, Arnhem, Gelderland",51.9400,5.9038,10,"flight call, nocturnal flight call",//www.xeno-canto.org/505641,//www.xeno-canto.org/505641/download,"XC505641-Koperwiek, Merel, Grote Lijster, 2310...",//creativecommons.org/licenses/by-nc-nd/4.0/,A,0:26,22:27,2019-10-23,2019-11-04,"['Turdus merula', 'Turdus iliacus']","a very rarely heard thrush, altough common at ...",no,no,//www.xeno-canto.org/sounds/uploaded/RNMRWBXEZ...,//www.xeno-canto.org/sounds/uploaded/RNMRWBXEZ...,//www.xeno-canto.org/sounds/uploaded/RNMRWBXEZ...,//www.xeno-canto.org/sounds/uploaded/RNMRWBXEZ...
6747,337115,Turdus viscivorus,,Turdus,viscivorus,NaN,Mistle Thrush,Nick Komar,Germany,"Hamburg, Hamburg",53.5518,9.9732,20,call,//www.xeno-canto.org/337115,//www.xeno-canto.org/337115/download,XC337115-GER Mistle Thrush rattle 0730 160915 ...,//creativecommons.org/licenses/by-nc-sa/4.0/,B,0:26,07:30,2016-09-15,2016-10-05,[''],Urban gardens at Planten in Blomen Park. Recor...,yes,no,//www.xeno-canto.org/sounds/uploaded/ODAYZINCO...,//www.xeno-canto.org/sounds/uploaded/ODAYZINCO...,//www.xeno-canto.org/sounds/uploaded/ODAYZINCO...,//www.xeno-canto.org/sounds/uploaded/ODAYZINCO...
6803,216272,Turdus viscivorus,,Turdus,viscivorus,NaN,Mistle Thrush,Frank Holzapfel,Germany,"Naturpark Stromberg-Heuchelberg, Baden-Württem...",49.0164,9.0228,360,song,//www.xeno-canto.org/216272,//www.xeno-canto.org/216272/download,XC216272-150306MisteldrosselGesangFH8edit.mp3,//creativecommons.org/licenses/by-nc-nd/4.0/,B,6:10,?,2015-03-06,2015-03-10,[''],Afternoon in mixed forest.,no,no,//www.xeno-canto.org/sounds/uploaded/NKZQLCJFU...,//www.xeno-canto.org/sounds/uploaded/NKZQLCJFU...,//www.xeno-canto.org/sounds/uploaded/NKZQLCJFU...,//www.xeno-canto.org/sounds/uploaded/NKZQLCJFU...
6418,551453,Turdus viscivorus,,Turdus,viscivorus,NaN,Mistle Thrush,Rolf A. de By,Netherlands,"Elferinkshave, Haaksbergen, Overijssel",52.1219,6.8237,40,"flight call, song",//www.xeno-canto.org/551453,//www.xeno-canto.org/551453/download,XC551453-rdbpara_2020-04-27_07h14_MistleThrush...,//creativecommons.org/licenses/by-nc-sa/4.0/,A,0:42,07:14,2020-04-27,2020-04-27,"['Fringilla coelebs', 'Erithacus rubecula', 'C...",NaN,yes,no,//www.xeno-canto.org/sounds/uploaded/PTYTXQRLS...,//www.xeno-canto.org/sounds/uploaded/PTYTXQRLS...,//www.xeno-canto.org/sounds/uploaded/PTYTXQRLS...,//www.xeno-canto.org/sounds/uploaded/PTYTXQRLS...
6624,557937,Turdus viscivorus,,Turdus,viscivorus,NaN,Mistle Thrush,Olivier SWIFT,France,"Saint-Servant, Morbihan, Bretagne",47.9279,-2.5175,40,"call, song",//www.xeno-canto.org/557937,//www.xeno-canto.org/557937/download,XC557937-TURVIS_191227_5306_StGobrien56.mp3,//creativecommons.org/licenses/by-nc-sa/4.0/,B,0:31,10:37,2019-12-27,2020-05-13,"['Turdus philomelos', 'Erithacus rubecula', 'T...",Tascam DR05 and Wildtronics parabole,yes,no,//www.xeno-canto.org/sounds/uploaded/JCPKCBKKA...,//www.xeno-canto.org/sounds/uploaded/JCPKCBKKA...,//www.xeno-canto.org/sounds/uploaded/JCPKCBKKA...,//www.xeno-canto.org/sounds/uploaded/JCPKCBKKA...


In [61]:
# train and dev dataset split 
X_train, X_test, y_train, y_test = model_selection.train_test_split(rand_id[['id', 'species', 'file', 'file-name', 'q', 'length']], rand_id['species'], test_size=0.33) 

Ci-dessous, l'objectif était de écrire 'train' ou 'test' dans la colonne  full_xeno [ 'train/dev_set' ] , en faisant une jointure avec X_train [ 'id' ] 

In [79]:
full_xeno[full_xeno['id'] == X_train['id']]  #['train/dev_set'] = 'train'

ValueError: ignored

In [93]:
full_xeno.loc[full_xeno['id'].isin(X_train.loc['id'])]['train/dev_set'] = 'train'

KeyError: ignored

In [97]:
# Downloading train examples
for id in full_xeno[full_xeno['train/dev_set'] == 'train']['id'] :
  bird = full_xeno.loc['species'].replace(" ", "_").lower()
  filepath = full_xeno[full_xeno['id'] == id]['file']
  filename = full_xeno[full_xeno['id'] == id]['file-name']

 # create folder if it doesn't exist
  dirPath = f'/content/drive/My Drive/xenocanto_data/train/{bird}'
    
  if not os.path.exists(dirPath):
      os.mkdir(dirPath)
    
    # retrieve recordings
  rec_path = f"https:{filepath}"
       
  urllib.request.urlretrieve(rec_path, f"/content/drive/My Drive/xenocanto_data/train/{bird}/{filename}")

In [ ]:
# Downloading test examples
for id in full_xeno[full_xeno['train/dev_set'] == 'test']['id'] :
  bird = full_xeno.loc['species'].replace(" ", "_").lower()
  filepath = full_xeno[full_xeno['id'] == id]['file']
  filename = full_xeno[full_xeno['id'] == id]['file-name']

 # create folder if it doesn't exist
  dirPath = f'/content/drive/My Drive/xenocanto_data/test/{bird}'
    
  if not os.path.exists(dirPath):
      os.mkdir(dirPath)
    
    # retrieve recordings
  rec_path = f"https:{filepath}"
       
  urllib.request.urlretrieve(rec_path, f"/content/drive/My Drive/xenocanto_data/test/{bird}/{filename}")